# Tech Intro

## Main tables

In [154]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [155]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[3] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [156]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1119 14:30:19.210762 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1119 14:30:19.211375  7691 sched.cpp:241] Version: 1.11.0
I1119 14:30:19.211593 10542 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1119 14:30:19.211638 10542 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1119 14:30:19.213004 10549 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75341


## Read the spec-file from Hadoop and show some data sample

In [157]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+----+----+-----------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Show columns' properties

In [158]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- body: string (nullable = true)
 |-- from: string (nullable = true)
 |-- date: string (nullable = true)
 |-- udate: string (nullable = true)
 |-- status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- read_by: string (nullable = true)
 |-- lead_id: string (nullable = true)
 |-- filial_id: string (nullable = true)



## Total rows (count)

In [159]:
eda_df.count()

9250

## Simple func to execute and show the result of the SQL-instruction

In [160]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [161]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [163]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|9250    |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [164]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,uid,subject,body,from,date,udate,status,source_id,read_by,lead_id,filial_id
0,0,0,0,65,0,0,9250,0,9250,9250,87,0


In [165]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,uid,subject,body,from,date,udate,status,source_id,read_by,lead_id,filial_id
0,0,0,0,0,0,0,0,9250,0,0,0,0


In [166]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,uid,subject,body,from,date,udate,status,source_id,read_by,lead_id,filial_id
0,0,0,58,0,0,0,0,0,0,0,0,0


In [167]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
uid,0,0,0
subject,0,0,58
body,65,0,0
from,0,0,0
date,0,0,0
udate,9250,0,0
status,0,9250,0
source_id,9250,0,0
read_by,9250,0,0


In [168]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [169]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.0,0.000
uid,0.000,0.0,0.000
subject,0.000,0.0,0.627
body,0.703,0.0,0.000
from,0.000,0.0,0.000
date,0.000,0.0,0.000
udate,100.000,0.0,0.000
status,0.000,100.0,0.000
source_id,100.000,0.0,0.000
read_by,100.000,0.0,0.000


### SQL-instructions

In [170]:
#Find dupes or misunderstanding or useless static values 9250 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|9250              |
+------------------+

+-------------------+
|count(DISTINCT uid)|
+-------------------+
|9243               |
+-------------------+

+-----------------------+
|count(DISTINCT subject)|
+-----------------------+
|864                    |
+-----------------------+



+--------------------+
|count(DISTINCT body)|
+--------------------+
|9151                |
+--------------------+

+--------------------+
|count(DISTINCT from)|
+--------------------+
|77                  |
+--------------------+

+--------------------+
|count(DISTINCT date)|
+--------------------+
|9242                |
+--------------------+

+---------------------+
|count(DISTINCT udate)|
+---------------------+
|0                    |
+---------------------+

+----------------------+
|count(DISTINCT status)|
+----------------------+
|1                     |
+----------------------+

+-------------------------+
|count(DISTINCT source_id)|
+-------------------------+
|0                        |
+-------------------------+

+-----------------------+
|count(DISTINCT read_by)|
+-----------------------+
|0                      |
+-----------------------+

+-----------------------+
|count(DISTINCT lead_id)|
+-----------------------+
|9163                   |
+-----------------------+

+-

In [177]:
#Find dupes or misunderstanding or useless static values 9250 total

execute("SELECT DISTINCT(subject)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY subject ASC")

execute("SELECT DISTINCT(from)\
        FROM eda_sql\
        WHERE 1 = 1 \
        ORDER BY from ASC")


+---------------------------------------------------------------+
|subject                                                        |
+---------------------------------------------------------------+
|                                                               |
| Изменения в Условиях использования YouTube                    |
|50% off on your subscription this Halloween                    |
|7 возможностей Почты Mail.ru, о которых стоит узнать          |
|A Little Gift from Avocode ❤️                                  |
|Abstract, Avocode, InVision, or Zeplin?                        |
|Are you working on web or mobile apps?                         |
|Avocode August Updates                                         |
|Avocode News - August                                          |
|Avocode News - September                                       |
|Avocode plugin for designing with real copy                    |
|Black Friday продолжается!                                     |
|Callback 

## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [178]:
spark.stop()

I1119 14:40:43.311687 10390 sched.cpp:2174] Asked to stop the driver
I1119 14:40:43.311825 10550 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75341
